In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
_4dm4_data_fp = 'dataset/4dm4.db'

_4dm4_connection = sqlite3.connect(_4dm4_data_fp)

In [3]:
ranked_loved_data_fp = "dataset/rankedscores_top_10000.db"

ranked_loved_connection = sqlite3.connect(ranked_loved_data_fp)

In [4]:
def get_all_4k_scores():
    sql = """
    SELECT osu_scores_mania_high.user_id, osu_scores_mania_high.beatmap_id, max(osu_scores_mania_high.pp)
    FROM osu_scores_mania_high
    WHERE osu_scores_mania_high.beatmap_id in (SELECT beatmap_id FROM osu_beatmaps 
    WHERE osu_beatmaps.diff_size = 4 AND osu_beatmaps.playmode = 3 AND osu_beatmaps.approved = 1)
    GROUP BY osu_scores_mania_high.user_id, osu_scores_mania_high.beatmap_id
    ORDER BY osu_scores_mania_high.user_id, max(osu_scores_mania_high.pp) DESC;
    """
    return pd.read_sql(sql, ranked_loved_connection).fillna(0)

In [5]:
all_4k_scores = get_all_4k_scores()

In [6]:
all_4k_scores[all_4k_scores['user_id'] == 10879600].sort_values('max(osu_scores_mania_high.pp)', ascending=False).count()

user_id                          545
beatmap_id                       545
max(osu_scores_mania_high.pp)    545
dtype: int64

In [7]:
def get_pp(player_id):
    top_pp = all_4k_scores[all_4k_scores['user_id'] == player_id].sort_values('max(osu_scores_mania_high.pp)', ascending=False)
    top_pp = top_pp['max(osu_scores_mania_high.pp)']
    weightage = np.power(0.95, np.arange(len(top_pp)))
    return float(np.dot(weightage, top_pp))

In [8]:
player_id_query = "SELECT user_id FROM osu_user_stats_mania"

player_ids = pd.read_sql(player_id_query, ranked_loved_connection)

In [9]:
player_ids

,user_id
0,61791
1,64811
2,65724
3,75777
4,82751
...,...
9995,29921432
9996,29963343
9997,30041483
9998,30058767


In [10]:
all_4k_scores

,user_id,beatmap_id,max(osu_scores_mania_high.pp)
0,61791,1967430,194.8160
1,61791,784054,191.0240
2,61791,484669,181.5850
3,61791,674460,173.9180
4,61791,1092805,172.6980
...,...,...,...
6240428,30146293,1001780,362.4250
6240429,30146293,1243559,361.2080
6240430,30146293,1091460,213.6920
6240431,30146293,770126,187.1050


In [11]:
decay_rate = 0.95

In [12]:
current_weight = 1

weights = []
current_user = 0
for user_id in all_4k_scores['user_id']:
    if user_id != current_user:
        current_user = user_id
        current_weight = 1
    weights.append(current_weight)
    current_weight *= decay_rate

In [13]:
all_4k_scores['weights'] = weights

In [14]:
all_4k_scores

,user_id,beatmap_id,max(osu_scores_mania_high.pp),weights
0,61791,1967430,194.8160,1.000000
1,61791,784054,191.0240,0.950000
2,61791,484669,181.5850,0.902500
3,61791,674460,173.9180,0.857375
4,61791,1092805,172.6980,0.814506
...,...,...,...,...
6240428,30146293,1001780,362.4250,0.397214
6240429,30146293,1243559,361.2080,0.377354
6240430,30146293,1091460,213.6920,0.358486
6240431,30146293,770126,187.1050,0.340562


In [15]:
all_4k_scores['weighted_pp'] = all_4k_scores['max(osu_scores_mania_high.pp)'] * all_4k_scores['weights']

In [16]:
_4k_pp = all_4k_scores[['user_id', 'weighted_pp']].groupby('user_id').sum()

In [20]:
_4k_pp = _4k_pp.sort_values('weighted_pp', ascending=False)

_4k_pp.head(25)

,weighted_pp,rank
user_id,,
10072733,14144.452552,3464
16734203,13787.054746,7147
2288363,13483.935500,386
15806513,13432.006374,6641
10083439,13357.408944,3471
10790649,13237.780362,3928
13385865,13195.042332,5475
7794488,12909.310253,2202
9169747,12896.345271,2941


In [21]:
_4k_pp['rank'] = np.arange(len(_4k_pp)) + 1

In [22]:
_4k_pp.to_csv('4kpp_estimate.csv')